In [1]:
%config IPCompleter.greedy=True  # Podpowiedzi składni

In [2]:
file = open('kroA100.tsp.txt')

In [3]:
tab = []

In [4]:
info = file.readline()[0:-1]

In [5]:
while info[0] != "1":
    info = file.readline()[0:-1]

In [6]:
print(info)

1 1380 939


In [7]:
while info!= "EOF":
    tab.append(info.split(" "))
    info = file.readline()[0:-1]

In [8]:
#print(type(info))

In [9]:
for x in range(len(tab)):
    for y in range(len(tab[0])):
        tab[x][y] = int(tab[x][y])

In [12]:
print(len(tab))

100


In [11]:
#print(type(tab[0][0]))

In [15]:
route_tab = [100][100]
for x in range(len(route_tab)):
    for y in range(len(route_tab[0])):
        i = 2

IndexError: list index out of range